In [1]:
import time
import numpy as np
from dotmap import DotMap

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.parameter import Parameter

import sys
sys.path.append('src')
from read import *
from utils import *
from metrics import *
from model import *

2024-07-16 19:36:55.930624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 19:36:56.800572: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-07-16 19:36:56.800645: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-07-16 19:36:56.800651: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
name = 'ml'
device = 'cuda:0'
cfg = DotMap()
cfg.dataset = name
cfg.device = device
cfg.rpath = './datasets'
cfg.neg_sampling = 'neg_sample0'

In [4]:

cfg.rank = 32
cfg.layer_dims = [3, 32, 1]
cfg.depth = len(cfg.layer_dims)
cfg.lr = 1e-3
cfg.wd = 1e-4
cfg.dropout =  0.8
cfg.dropout2 =  0.2
epochs = 1000
cfg.batch_size = 1024


In [6]:
tensor= read_data(cfg)
cfg.sizes = tensor.sizes
dataset = COODataset(tensor.train_i, tensor.train_v)
dataloader = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True)

DATASET: ml || size: [610, 9724, 4110] & training nnz: 161336


In [7]:
# create the model
model = NeAT(cfg, cfg.sizes).to(device)
loss_fn = nn.BCELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.wd)
m = nn.Sigmoid()

In [13]:
flag = 0
err_lst = []
old_valid_acc = 1e+6
# train the model
for epoch in range(epochs):
    start = time.time()
    model.train()
    epoch_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        inputs, targets = batch[0], batch[1]
        outputs = model(inputs)
        loss = loss_fn(m(outputs), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    model.eval()
    end = time.time()
    if (epoch+1) % 1 == 0:
        with torch.no_grad():
            train_rec = m(model(tensor.train_i))
            val_rec = m(model(tensor.valid_i))
            r = eval_(val_rec.data, tensor.valid_v)
            print(f"Epochs {epoch} {end-start:.2}s elapsed || "
                    f"Acc:{r['acc']:.4f} Recall:{r['recall']:.4f}"
                    f"Prec.:{r['prec']:.4f} F1:{r['f1']:.4f} AUC : {r['auc']:.4f}")

            if (old_valid_acc > r['acc']):
                flag +=1
            if flag == 5:
                break
            old_valid_acc = r['acc']

        # err_lst.append([train_rmse.item(), valid_rmse.item()])
    

Epochs 0 1.6s elapsed || Acc:0.9816 Recall:0.9741Prec.:0.9888 F1:0.9814 AUC : 0.9984
Epochs 1 1.5s elapsed || Acc:0.9830 Recall:0.9779Prec.:0.9881 F1:0.9830 AUC : 0.9984
Epochs 2 1.9s elapsed || Acc:0.9835 Recall:0.9779Prec.:0.9891 F1:0.9834 AUC : 0.9985
Epochs 3 1.5s elapsed || Acc:0.9832 Recall:0.9776Prec.:0.9887 F1:0.9831 AUC : 0.9985
Epochs 4 1.4s elapsed || Acc:0.9834 Recall:0.9793Prec.:0.9874 F1:0.9833 AUC : 0.9985
Epochs 5 1.5s elapsed || Acc:0.9830 Recall:0.9769Prec.:0.9890 F1:0.9829 AUC : 0.9985
Epochs 6 1.6s elapsed || Acc:0.9844 Recall:0.9813Prec.:0.9874 F1:0.9843 AUC : 0.9985
Epochs 7 1.3s elapsed || Acc:0.9838 Recall:0.9793Prec.:0.9882 F1:0.9837 AUC : 0.9984
Epochs 8 1.6s elapsed || Acc:0.9836 Recall:0.9788Prec.:0.9883 F1:0.9835 AUC : 0.9985
